<font color="#138D75">**NERO Winter School training**</font> <br>
**Copyright:** (c) 2025 EUMETSAT <br>
**License:** GPL-3.0-or-later <br>
**Authors:** Andrea Meraner (EUMETSAT), based on <a href='https://firms.modaps.eosdis.nasa.gov/content/academy/data_api/firms_api_use.html'>example script on the FIRMS documentation</a>.
# Access and Process Active Fires/Thermal Anomalies Products from NASA FIRMS to shapefiles

## Intro

NASA FIRMS (Fire Information for Resource Management System) provides access to near real-time data related to wildfire activity from a range of instruments and platforms. We will use the FIRMS API to retrieve thermal anomalies products, in form of point data inside a shapefile, for MODIS (onboard Terra and Aqua) and VIIRS (onboard Suomi-NPP, NOAA-20 and NOAA-21).

## Learning Objectives

- How to access and extract active fire point data from the FIRMS API.
- How to store point data in a shapefile

## Links and Resouces
- FIRMS Website: https://firms.modaps.eosdis.nasa.gov/
- FIRMS Global Map, displays hotspots and imagery: https://firms.modaps.eosdis.nasa.gov/map/
- FAQ page with useful information on products: https://www.earthdata.nasa.gov/data/tools/firms/faq
- Download page with several download options in different formats, e.g. with aggregated data: https://firms.modaps.eosdis.nasa.gov/download/
- API Documentation with further code examples: https://firms.modaps.eosdis.nasa.gov/api/

<hr>

## Before we start
To run this script you need to retrieve your own MAP_KEY to authenticate yourself on the FIRMS API here https://firms.modaps.eosdis.nasa.gov/api/map_key/

After you got your own key, you have to open the credentials.py file and input it in the variable `FIRMS_MAP_KEY`

<hr>

### Setup
Import the required packages and the credentials from credentials.py

In [1]:
import os

import geopandas as gpd
import pandas as pd

import credentials

Configure your query by defining the time and area. Time should be a day in the format `"2024-09-15"` and the latitude-longitude bounding box `lonlat_bbox` is generated from the `W`est, `S`outh, `E`ast, `N`orth variables. The `run_name` is used to create a subfolder in `output_folder` containing the results.

In [2]:
day = "2024-09-15"

output_folder = "./test/"
run_name = "testrun"

W = 26.5
S = 41.7
E = 27.3
N = 42.3
lonlat_bbox = [W, S, E, N]

### Access data
Now choose the instrument you want to retrieve the data for. You can pick between following options:
- MODIS_NRT or MODIS_SP
- VIIRS_SNPP_NRT or VIIRS_SNPP_SP
- VIIRS_NOAA20_NRT
- VIIRS_NOAA21_NRT

The MODIS options will combine data from both Terra (morning pass) and Aqua (afternoon pass) platforms. For VIIRS you can choose the platform between SNPP, NOAA-20 and NOAA-21.
For MODIS and VIIRS_SNPP there are the NRT and SP options: NRT (near-real-time) are products that are generated rapidly after the acquisition to support fire monitoring activities; SP (Standard Products) are reprocessed products of higher quality, particularly regarding the geolocation accurancy of fires. These replace and superseed the NRT products when made available. More information is available e.g. here: https://www.earthdata.nasa.gov/data/tools/firms/faq under _Getting Started -> What are the key differences between URT/RT/NRT and Standard quality fire data?_.

Enter the product you chose in the field below and run the cell to query the FIRMS API using the custom URL and Pandas. The obtained DataFrame will be shown. If the result is empty, try another instrument, product type or widen/change your search parameters.


In [4]:
instrument = "MODIS_NRT"  # <-- input your instrument product choice

area_url = (f'https://firms.modaps.eosdis.nasa.gov/api/area/csv/{credentials.FIRMS_MAP_KEY}/'
            f'{instrument}/'
            f'{lonlat_bbox[0]},{lonlat_bbox[1]},{lonlat_bbox[2]},{lonlat_bbox[3]}/1/{day}')
df = pd.read_csv(area_url)

df  # let's look at the data

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,40.66123,-8.48624,327.66,1.37,1.16,2024-09-15,1403,Aqua,MODIS,78,6.1NRT,303.82,29.63,D
1,40.66164,-8.49763,317.47,1.37,1.16,2024-09-15,1403,Aqua,MODIS,57,6.1NRT,303.22,13.23,D
2,40.66456,-8.48201,327.76,1.37,1.16,2024-09-15,1403,Aqua,MODIS,78,6.1NRT,303.67,29.99,D
3,40.67025,-8.60609,317.41,1.39,1.17,2024-09-15,1403,Aqua,MODIS,46,6.1NRT,303.91,13.12,D


### Process data and save to shapefile
Once you found a query that returns some results, let's run the next cells to put the data into a shapefile ready to be opened in QGIS. The code comments contain some information on the processing.

In [5]:
# convert the pandas DataFrame to a geopandas GeoDataFrame to be able to perform geometric operations and exports
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']), crs="EPSG:4326")
# Set the CRS to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)
# merge the fields acq_date and acq_time to produce a day_time column for easier categorisation in QGIS
gdf['day_time'] = (gdf['acq_date'] + ' ' + gdf['acq_time'].astype(str).str.zfill(4).str[:2]
                   + ':' + gdf['acq_time'].astype(str).str.zfill(4).str[2:])

gdf  # let's look at the table before saving it

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,geometry,day_time
0,40.66123,-8.48624,327.66,1.37,1.16,2024-09-15,1403,Aqua,MODIS,78,6.1NRT,303.82,29.63,D,POINT (-8.48624 40.66123),2024-09-15 14:03
1,40.66164,-8.49763,317.47,1.37,1.16,2024-09-15,1403,Aqua,MODIS,57,6.1NRT,303.22,13.23,D,POINT (-8.49763 40.66164),2024-09-15 14:03
2,40.66456,-8.48201,327.76,1.37,1.16,2024-09-15,1403,Aqua,MODIS,78,6.1NRT,303.67,29.99,D,POINT (-8.48201 40.66456),2024-09-15 14:03
3,40.67025,-8.60609,317.41,1.39,1.17,2024-09-15,1403,Aqua,MODIS,46,6.1NRT,303.91,13.12,D,POINT (-8.60609 40.67025),2024-09-15 14:03


In [9]:
# Save all columns as properties in a shapefile
output_filename = f"{day}_firms_thermal_anomalies_{instrument}.shp"
shapefile_path = os.path.join(output_folder, run_name, 'Satellite_ActiveFires', 'MODIS-VIIRS', output_filename)
os.makedirs(os.path.dirname(shapefile_path), exist_ok=True)
gdf.to_file(shapefile_path, driver='ESRI Shapefile')
print(f"Saved {shapefile_path}")

Saved ./test/testrun/2024-09-15_firms_thermal_anomalies_MODIS_NRT.shp


## Automated extraction for a range of days (optional)
In the cell below, define a start and end day in the format `2024-09-15` to run the automated extraction of data for the define lat-lon bounding box for all instruments, by calling the `main_firms` function. The function essentially contains the same code as above, but with iterative loops to process each day and instrument sequentially. Files from all available MODIS and VIIRS products will be combined into one file each.

In [7]:
from firms_script import main_firms

start_day = "2024-09-15"
end_day = "2024-09-20"

main_firms(start_day, end_day, lonlat_bbox, output_folder, run_name)


Querying FIRMS for day 2024-09-15
Searching for MODIS data...
Found 4 data points for https://firms.modaps.eosdis.nasa.gov/api/area/csv/ff70b2bdd77dbf08d9036ca41a6d5411/MODIS_NRT/-8.75,40.52,-8.45,40.72/1/2024-09-15
No data found for https://firms.modaps.eosdis.nasa.gov/api/area/csv/ff70b2bdd77dbf08d9036ca41a6d5411/MODIS_SP/-8.75,40.52,-8.45,40.72/1/2024-09-15
--> Found 4 data points for MODIS on 2024-09-15. Exporting.
Saved ./test/testrun/2024-09-15_firms_active_fires_MODIS.shp
Searching for VIIRS data...
No data found for https://firms.modaps.eosdis.nasa.gov/api/area/csv/ff70b2bdd77dbf08d9036ca41a6d5411/VIIRS_SNPP_NRT/-8.75,40.52,-8.45,40.72/1/2024-09-15
Found 2 data points for https://firms.modaps.eosdis.nasa.gov/api/area/csv/ff70b2bdd77dbf08d9036ca41a6d5411/VIIRS_NOAA20_NRT/-8.75,40.52,-8.45,40.72/1/2024-09-15
Found 3 data points for https://firms.modaps.eosdis.nasa.gov/api/area/csv/ff70b2bdd77dbf08d9036ca41a6d5411/VIIRS_NOAA21_NRT/-8.75,40.52,-8.45,40.72/1/2024-09-15
Found 1 data